<a href="https://colab.research.google.com/github/head1ton/llm_a_z_finetuning/blob/main/notebook/keyword_search_ensemble_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BM25 알고리즘은 사용자가 특정 단어를 검색했을 때, 어떤 문서가 더 적합한지 문서의 순위를 결정하는 알고리즘입니다.

**단어 빈도**  
사용자가 「인공지능」이라는 단어를 검색했다고 생각해봅시다. 문서가 두 개 있는데, 첫 번째 문서에는 「인공지능」이 여러 번 반복해서 등장하고, 두 번째 문서에는 딱 한 번만 등장했다고 합시다. 그렇다면 우리는 첫 번째 문서가 더 관련이 깊다고 생각하게 됩니다. BM25 알고리즘 역시 이러한 판단을 합니다. 이것을 **단어 빈도(Term Frequency, TF)**라고 합니다. 특정 단어가 문서 내에 더 자주 나타날수록 해당 문서에서 그 단어는 중요한 역할을 한다고 판단합니다.

**역문서 빈도**  
하지만 여기서 끝이 아닙니다. 만약 모든 문서에 너무 흔하게 나타나는 「그리고」, 「또한」 같은 단어라면 어떨까요? 이런 단어들은 아무리 문서 내에서 빈도가 높아도 특정 문서의 중요도를 판단하는 데 도움이 되지 않습니다. 반면 「인공지능」, 「머신러닝」, 「딥러닝」과 같은 특정 주제를 나타내는 단어들은 모든 문서에 흔히 등장하지 않고, 일부 문서에만 특별히 등장합니다. 이런 희귀한 단어들은 문서의 주제를 보다 명확하게 드러내는 단서가 되므로, BM25는 이런 단어에 더 높은 점수를 부여합니다. 이것이 바로 BM25가 중요하게 고려하는 두 번째 요소, **역문서 빈도(Inverse Document Frequency, IDF)**입니다.

역문서 빈도(IDF)를 조금 더 쉽게 표현하면, 특정 단어가 전체 문서 집합에서 얼마나 「희귀한지」를 수치화한 것입니다. IDF의 공식은 다음과 같습니다.

$$
\text{IDF} = \log\frac{N - n + 0.5}{n + 0.5}
$$

각 항목을 쉽게 설명하면 다음과 같습니다.

- $N$ : 전체 문서의 개수입니다.
- $n$ : 특정 단어가 나타나는 문서의 개수입니다.

즉, 특정 단어가 전체 문서 중에서 적은 수의 문서에서만 나타날수록, 다시 말해 더 희귀한 단어일수록 IDF 값은 높아지고, 이 단어가 해당 문서에 등장할 때의 중요도를 더 높게 판단하게 됩니다.

**문서 길이**  
마지막으로, BM25는 문서 길이라는 요소까지 고려하여 점수를 조정합니다. 같은 횟수로 등장한 단어라도 문서 길이에 따라 그 중요성이 달라지기 때문입니다. 예를 들어, 5000개의 단어로 이루어진 긴 문서에 「인공지능」이 5번 등장한 것과, 500개의 단어로 이루어진 짧은 문서에 같은 「인공지능」이 5번 등장한 것은 의미가 다릅니다. 긴 문서에서는 같은 횟수라도 그 중요성이 상대적으로 떨어질 수밖에 없습니다. BM25는 문서 길이를 기준으로, 긴 문서에 등장한 빈도의 가치를 낮추고, 짧은 문서에 등장한 빈도의 가치를 높이는 보정을 수행합니다.

**세가지 요소를 결합한 공식**

지금까지 설명한 세 가지 요소, 즉  
① 단어의 빈도(TF),  
② 단어가 얼마나 희귀한지(IDF),  
③ 문서의 길이  
를 모두 반영한 최종적인 BM25 공식은 다음과 같습니다.

$$
\text{BM25 점수} = \text{IDF} \times \frac{f \times (k_1 + 1)}{f + k_1 \times \left(1 - b + b \times \frac{L}{\text{avgL}}\right)}
$$

각 항목의 의미를 다시 간단히 정리하면 다음과 같습니다.

- $f$ : 특정 단어가 문서에서 등장한 빈도(횟수)
- $L$ : 문서의 길이(문서가 가진 전체 단어 수)
- $avgL$ : 문서 전체의 평균 길이
- $k_1$, $b$ : 빈도와 문서 길이를 조정하는 상수 (일반적으로 $k_1 = 1.2 \sim 2.0$, $b = 0.75$ 정도로 사용)
- IDF : 앞서 설명한 단어의 희귀성 지표(역문서 빈도)

정리하면, BM25는 사용자가 검색한 「인공지능」이라는 단어가 특정 문서 내에서 많이 등장할수록(빈도가 높을수록), 전체 문서 중 매우 적은 수의 문서에만 나타나는 희귀한 단어일수록(IDF가 클수록), 그리고 문서의 길이가 짧을수록(L이 avgL보다 작을수록), 더 높은 점수를 주어 해당 문서를 사용자가 원하는 검색 결과의 상위에 배치합니다.

In [1]:
!pip -q install langchain openai tiktoken langchain-community rank_bm25 sentence-transformers chromadb pypdf langchain-huggingface langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.9/475.9 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 121.1 MB/s eta 0

In [2]:
import tiktoken
import openai
from typing import List, Dict, Optional, Any
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma, FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader, WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers  import BM25Retriever

In [3]:
!wget https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf

--2025-12-14 21:18:11--  https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf
Resolving wdr.ubion.co.kr (wdr.ubion.co.kr)... 61.100.182.43
Connecting to wdr.ubion.co.kr (wdr.ubion.co.kr)|61.100.182.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253369 (1.2M) [application/pdf]
Saving to: ‘gsat_170823.pdf’

gsat_170823.pdf     100%[===================>]   1.19M   440KB/s    in 2.8s    

2025-12-14 21:18:15 (440 KB/s) - ‘gsat_170823.pdf’ saved [1253369/1253369]



In [4]:
## pdf 파일로드 하고 쪼개기
loader = PyPDFLoader('https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf')
pages = loader.load_and_split()
print(len(pages))

27


In [5]:
model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Document(metadata={이 파일의 정보}, page_content='내용')

In [6]:
pages[0].page_content

'2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등\nIM HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등\nDS DRAM, NAND Flash, 모바일AP, LCD패널, OLED패널, LED 등 \n∙ 주요 사업은 전자전지기계 등 제조, 전자통신기 등 제도, 컴퓨터 등 제조, 반도체 제조·조립 등. 주요 \n제품과 구성비율은 HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등 54%, TV, 모니터, 냉장고, 세탁기, \n에어컨 등 2 4 % ,  D R A M ,  N A N D  F l a s h ,  모바일 AP 등 19% 등으로 구분 \n∙ 지역별로는 본사를 거점으로 한국 및 CE, IM 부문 산하 해외 9개 지역총괄과 DS 부문 산하 해외 5개 \n지역총괄의 생산ㆍ판매법인 등 165개의 동종업종을 영위하는 종속기업으로 구성\n∙ 국내사업장 : 수원, 구미, 기흥, 화성, 온양, 광주\n∙ 해외사업장 : 86개국 220개지점 (15년 말 기준)\n1) 연결재무재표 기준\n2) 연결재무재표 기준'

In [7]:
pages[1].page_content

'3\n2) 주요 사업 분야 \n⑴ CE(Consumer Electronics)부문 \n① 산업의 특성 \n∙TV산업은 1926년 흑백 TV 개발, 1954년 RCA사가 Color TV(21") 양산/판매를 시작한 이래로 트리니트\n론 브라운관(1967년), 완전평면 브라운관(1996년) 개발 등 기술적인 발전을 거듭\n∙ 주요 국가 보급률이 90% 넘어서면서 브라운관 TV사업의 성장은 정체 \n∙ 그러나 하드웨어 측면에서의 Flat Panel TV(LCD, PDP) 출시, 소프트웨어 측면에서의 디지털 방송 \n확산(영/미 1998년~ )을 통해 TV 시장은 또 다시 강력한 성장 모멘텀을 되찾음 \n∙FPTV는 화질, 디자인 등 제품 성능의 향상과 지속적인 Set가격 하락으로 인해 성장을 지속하며 기존 \nCRT 시장을 빠르게 대체\n∙ 2010년 3D TV출시\n∙ 2011년~2012년 스마트TV 태동 \n∙ 2013년 UHD TV 출시 \n∙ 2014년 Curved TV 출시 \n∙2016년도 전체 TV 수요는 전년비 1.6% 하락한 2억 2,252만대로 제품별로는 LCD-TV(LED-TV포함)가 \n전년비 1.3% 하락한 2억 2,131만대 전망되나, 고해상도 Needs확대에 따라 UHD TV는 56백만대로 \n전년比 75% 성장할 것으로 예상(출처 : 2016.9월 IHS Report)\n② 국내외 시장연건 \n∙ TV시장의 Mega Trend인 대형화/고화질화가 Device간, 업체간 경쟁 격화에 따라 더욱 빠른 속도로 \n진행되고 있으며, 이에 따라 제품력과 브랜드파워를 앞세운 Major 업체의 강세가 지속\n∙ 고화질 및 슬림 제품에 대한 소비자 Needs가 높아짐에 따라 친환경 소재인 LED BLU(Back Light \nUnit)를 적용하여 TV의 밝기와 명암비를 높이고 소비전력을 낮춘 LED TV가 시장의 Main Stream으로 \n자리 잡음\n∙ 2010년에는 다양한 App을 이용할 수있는 스마트 TV를 출시하고 시장을 주도\n∙2012년

In [8]:
len(pages)

27

In [9]:
## chunk로 쪼개기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(pages)

In [10]:
len(texts)

69

In [11]:
len(texts[0].page_content)

490

## BM-25

In [12]:
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2

In [13]:
# BM25 리트리버만 사용하는 경우
docs = bm25_retriever.invoke("삼성전자의 사업 영역은?")

for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 10, 'page_label': '11'}
:
12 Q2 삼성전자의 Harman사 인수에 대해 어떻게 생각하십니까 ? A  삼성전자가 최근 80억달러(약 9조)에 미국의 전장업체(자동차 전자기기에 대한 사업)인 Harman 사를  인수하기로 결정했습니다. 하만사는 세계적으로 자동차용 인포테인먼트와 텔레매틱스 시장에서 점유 율 1~2위를 달리고 있으며 매출도 상당합니다. 이번 삼성전자의 하만 인수는 모바일 및 가전 시장에서  나아가 Connected Car 시장 진입에 진출했다는 것에 큰 의의가 있습니다. 향후 10년 시장 내 Connected  Car의 비중은 90%에 육박할 전망입니다. LG전자 역시 자동차부품(VC) 사업 부문에 있어 상당한 경쟁  위협 요인가 될 것으로 내다봤습니다. <관련기사> 전장사업 강자 하만과 손잡은 삼성…스마트카 ‘티어1’ 노린다 (한국경제. 2016-11-21) 삼성전자는 자동차 전장(電裝)사업에서 후발 주자다. 2005년 전장사업을 시작한 LG전자는
474
******************************
{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:0

In [ ]:
len(docs)

2

In [ ]:
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 10, 'page_label': '11'}
:
12 Q2 삼성전자의 Harman사 인수에 대해 어떻게 생각하십니까 ? A  삼성전자가 최근 80억달러(약 9조)에 미국의 전장업체(자동차 전자기기에 대한 사업)인 Harman 사를  인수하기로 결정했습니다. 하만사는 세계적으로 자동차용 인포테인먼트와 텔레매틱스 시장에서 점유 율 1~2위를 달리고 있으며 매출도 상당합니다. 이번 삼성전자의 하만 인수는 모바일 및 가전 시장에서  나아가 Connected Car 시장 진입에 진출했다는 것에 큰 의의가 있습니다. 향후 10년 시장 내 Connected  Car의 비중은 90%에 육박할 전망입니다. LG전자 역시 자동차부품(VC) 사업 부문에 있어 상당한 경쟁  위협 요인가 될 것으로 내다봤습니다. <관련기사> 전장사업 강자 하만과 손잡은 삼성…스마트카 ‘티어1’ 노린다 (한국경제. 2016-11-21) 삼성전자는 자동차 전장(電裝)사업에서 후발 주자다. 2005년 전장사업을 시작한 LG전자는
474
******************************
{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:0

## 임베딩

검색 대상 문서들을 임베딩 한 후에 적재하는 곳 => 벡터 데이터베이스  

랭체인을 통해서 우리가 사용할 수 있는 대표적인 벡터 데이터베이스: 크로마, 파이스

In [14]:
# 벡터 DB의 임베딩으로는 오픈소스 임베딩을 사용
chroma_vector = Chroma.from_documents(texts, model_huggingface)
chroma_retriever = chroma_vector.as_retriever(search_kwargs={'k':2})

In [15]:
# 크로마 리트리버만 사용하는 경우
docs = chroma_retriever.invoke("삼성전자의 사업 영역은?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'page_label': '10', 'moddate': '2017-08-16T00:21:02-08:00', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'page': 9, 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)'}
:
11 Ⅱ 기업 상세 분석 1  사업분야(내용) Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ? A  삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica- tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다. ⑴ Consumer Electronics (CE) 부문  ① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에  도전 ② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비 ③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발 ⑵ Information technology & Mobile communication (IM) 부문 ① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공
499
******************************
{'total_pages': 27, 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'creator': 'nPDF (pdftk 1.41)', 'moddate': '2017

## 앙상블

### EnsembleRetriever 작동 과정 (상세 예시)

먼저 두 검색기가 있고, 각각 k=4개의 문서를 반환한다고 가정해보겠습니다:

#### 1. 개별 검색기 결과 (각 검색기의 상위 4개 문서)
```
BM25 검색기 결과:
- 문서 A: 점수 0.9
- 문서 B: 점수 0.8
- 문서 C: 점수 0.7
- 문서 D: 점수 0.6

임베딩 검색기 결과:
- 문서 A: 점수 0.95 (BM25와 중복)
- 문서 E: 점수 0.85
- 문서 F: 점수 0.75
- 문서 B: 점수 0.65 (BM25와 중복)
```
여기서 문서 A와 B는 두 검색기 모두에서 반환되었습니다.[링크 텍스트](https://)

#### 2. 가중치 적용 (각 검색기에 0.5씩 가중치 부여)

각 검색기의 점수에 가중치 0.5를 곱합니다:
```
BM25 검색기:
- 문서 A: 0.9 × 0.5 = 0.45
- 문서 B: 0.8 × 0.5 = 0.40
- 문서 C: 0.7 × 0.5 = 0.35
- 문서 D: 0.6 × 0.5 = 0.30

임베딩 검색기:
- 문서 A: 0.95 × 0.5 = 0.475
- 문서 E: 0.85 × 0.5 = 0.425
- 문서 F: 0.75 × 0.5 = 0.375
- 문서 B: 0.65 × 0.5 = 0.325
```


#### 3. 중복 문서 점수 합산

동일한 문서가 여러 검색기에서 나온 경우, 가중치가 적용된 점수들을 합산합니다:
```
- 문서 A: 0.45 (BM25) + 0.475 (임베딩) = 0.925
- 문서 B: 0.40 (BM25) + 0.325 (임베딩) = 0.725
- 문서 C: 0.35 (BM25만) = 0.35
- 문서 D: 0.30 (BM25만) = 0.30
- 문서 E: 0.425 (임베딩만) = 0.425
- 문서 F: 0.375 (임베딩만) = 0.375
```


#### 4. 점수 기준 정렬 및 최종 결과 반환
```
모든 문서를 최종 점수 기준으로 내림차순 정렬하고, 상위 k개(예: k=4)를 반환합니다:

최종 정렬:
1. 문서 A: 0.925 (두 검색기 모두에서 높은 점수)
2. 문서 B: 0.725 (두 검색기 모두에서 등장)
3. 문서 E: 0.425 (임베딩 검색기에서만)
4. 문서 F: 0.375 (임베딩 검색기에서만)
5. 문서 C: 0.35 (BM25 검색기에서만)
6. 문서 D: 0.30 (BM25 검색기에서만)

최종 반환 결과(k=4): 문서 A, B, E, F
```

### 핵심 포인트
1. **중복 문서 강화**: 두 검색기 모두에서 나온 문서(A, B)는 점수가 합산되어 순위가 높아집니다. 이는 여러 방식으로 관련성이 확인된 문서가 우선시됨을 의미합니다.

2. **다양한 결과 포함**: 각 검색기의 고유한 강점을 활용하여 키워드 매칭(BM25)과 의미적 유사성(임베딩) 모두에서 관련성 높은 문서를 포함합니다.

3. **가중치 영향**: 만약 BM25에 더 높은 가중치(예: 0.7)를 부여한다면, BM25 결과가 최종 순위에 더 큰 영향을 미치게 됩니다.

**공지사항**
- 랭체인이 1.0으로 버전업되면서 기존 강의 녹화 시에 동작하던 앙상블 리트리버가 더 이상 제공되지 않습니다. 따라서 강사가 직접 앙상블 리트리버를 구현하였습니다.

In [16]:
class EnsembleRetriever:
    """
    BM25 + Dense retriever 앙상블.
    Reciprocal Rank Fusion (RRF) 알고리즘으로 점수 결합.
    """
    def __init__(self, retrievers: List, weights: Optional[List[float]] = None, c: int = 60, id_key: Optional[str] = None):
        assert len(retrievers) == len(weights), "retrievers와 weights 길이가 다릅니다."
        self.retrievers = retrievers
        self.weights = weights
        self.c = c
        self.id_key = id_key

    def _get_doc_id(self, doc: Document) -> str:
        """문서 중복 판단용 ID 추출"""
        if self.id_key and self.id_key in doc.metadata:
            return str(doc.metadata[self.id_key])
        return doc.page_content[:150]

    def _fuse_results(self, all_results: List[List[Document]]) -> List[Document]:
        """Reciprocal Rank Fusion 계산"""
        scores: Dict[str, Dict[str, Any]] = {}
        for i, docs in enumerate(all_results):
            weight = self.weights[i] if self.weights else 1.0
            for rank, doc in enumerate(docs):
                doc_id = self._get_doc_id(doc)
                score = weight / (rank + 1 + self.c)
                if doc_id not in scores:
                    scores[doc_id] = {"doc": doc, "score": score}
                else:
                    scores[doc_id]["score"] += score

        sorted_docs = sorted(scores.values(), key=lambda x: x["score"], reverse=True)
        return [item["doc"] for item in sorted_docs]

    def invoke(self, query: str, k: int = 4) -> List[Document]:
        """모든 retriever에서 검색 후 점수 융합"""
        all_results = []
        for retriever in self.retrievers:
            if hasattr(retriever, "get_relevant_documents"):
                docs = retriever.get_relevant_documents(query)
            elif hasattr(retriever, "invoke"):
                docs = retriever.invoke(query)
            else:
                raise AttributeError(f"{retriever} does not support retrieval.")
            all_results.append(docs[:k])

        fused_docs = self._fuse_results(all_results)
        return fused_docs[:k]

In [17]:
ensemble_retriever = EnsembleRetriever(
                    retrievers = [bm25_retriever, chroma_retriever], weights = [0.5,0.5])

docs = ensemble_retriever.invoke("삼성전자의 사업 영역은?")

In [18]:
# 문서의 개수는 총 4개이다.
len(docs)

4

## 앙상블 결과

In [19]:
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 10, 'page_label': '11'}
:
12 Q2 삼성전자의 Harman사 인수에 대해 어떻게 생각하십니까 ? A  삼성전자가 최근 80억달러(약 9조)에 미국의 전장업체(자동차 전자기기에 대한 사업)인 Harman 사를  인수하기로 결정했습니다. 하만사는 세계적으로 자동차용 인포테인먼트와 텔레매틱스 시장에서 점유 율 1~2위를 달리고 있으며 매출도 상당합니다. 이번 삼성전자의 하만 인수는 모바일 및 가전 시장에서  나아가 Connected Car 시장 진입에 진출했다는 것에 큰 의의가 있습니다. 향후 10년 시장 내 Connected  Car의 비중은 90%에 육박할 전망입니다. LG전자 역시 자동차부품(VC) 사업 부문에 있어 상당한 경쟁  위협 요인가 될 것으로 내다봤습니다. <관련기사> 전장사업 강자 하만과 손잡은 삼성…스마트카 ‘티어1’ 노린다 (한국경제. 2016-11-21) 삼성전자는 자동차 전장(電裝)사업에서 후발 주자다. 2005년 전장사업을 시작한 LG전자는
474
******************************
{'page': 9, 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'total_pages': 27, 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 

## 답변 얻기

**공지사항**
- 랭체인이 1.0으로 버전업되면서 더 이상 강의 녹화 시에 설명한 RetrievalQA.from_chain_type()가 지원되지 않습니다.  
따라서 아래 코드도 강사가 현재 버전에서 동작하도록 구현하였습니다.

In [ ]:
# from langchain_openai import ChatOpenAI
# from langchain_core.prompts import ChatPromptTemplate

# llm_model = ChatOpenAI(model_name='gpt-4o',
#                        api_key="여러분의 키 값",
#                        temperature=0)

In [25]:
# prompt = ChatPromptTemplate.from_template("""
# 아래의 문서를 참고하여 질문에 대한 답변을 작성하세요.

# 질문: {question}

# 참고 문서:
# {context}

# 답변:
# """)

In [20]:
import os
from typing import Literal

# from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# load_dotenv()

from google.colab import userdata


def _resolve_api_context() -> tuple[str, str]:
    """선택된 API 키와 베이스 URL 정보를 반환합니다."""
    # api_key = os.getenv("OPENROUTER_API_KEY")
    api_key = userdata.get('OPENROUTER_API_KEY')

    if not api_key:
        raise RuntimeError("OPENROUTER_API_KEY가 필요합니다.")

    base_url = os.getenv("OPENROUTER_API_BASE") or "https://openrouter.ai/api/v1"

    return (api_key, base_url)


def create_openrouter_llm(
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.3,
    max_tokens: int | None = None,
    **kwargs: object,
) -> ChatOpenAI:
    """OpenAI 호환 LLM 생성 헬퍼.

    Args:
        model: 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/gpt-4o, anthropic/claude-3-sonnet, google/gemini-pro)
        temperature: 생성 온도 (0.0-2.0)
        max_tokens: 최대 생성 토큰 수

    Returns:
        ChatOpenAI: 설정된 LLM 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    openai_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "temperature": temperature,
        "max_retries": 3,
        "timeout": 60,
        **kwargs,
    }
    if max_tokens is not None:
        openai_kwargs["max_tokens"] = max_tokens
    if base_url:
        openai_kwargs["base_url"] = base_url
    return ChatOpenAI(**openai_kwargs)

def create_embedding_model(
    model: str = "openai/text-embedding-3-small",
    **kwargs,
) -> OpenAIEmbeddings:
    """OpenAI 호환 임베딩 모델 생성.

    Args:
        model: 임베딩 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/text-embedding-3-small, openai/text-embedding-3-large)
        **kwargs: 추가 파라미터 (encoding_format 등은 model_kwargs로 전달됨)

    Returns:
        OpenAIEmbeddings: 설정된 임베딩 모델 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    # 전달받은 kwargs에서 model_kwargs로 전달할 파라미터 분리
    # encoding_format, extra_headers 등은 model_kwargs로 전달
    model_kwargs: dict = {}
    embedding_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "show_progress_bar": True,
        "skip_empty": True,
    }

    # 전달받은 kwargs 처리
    for key, value in kwargs.items():
        # OpenRouter API 특정 파라미터는 model_kwargs로 전달
        if key in ("encoding_format"):
            model_kwargs[key] = value
        else:
            # 나머지는 OpenAIEmbeddings에 직접 전달
            embedding_kwargs[key] = value

    if base_url:
        embedding_kwargs["base_url"] = base_url

    # model_kwargs가 있으면 전달
    if model_kwargs:
        embedding_kwargs["model_kwargs"] = model_kwargs

    return OpenAIEmbeddings(**embedding_kwargs)


def create_embedding_model_direct(
    model: str = "qwen/qwen3-embedding-0.6b",
    encoding_format: Literal["float", "base64"] = "float",
    input_text: str | list[str] = "",
    **kwargs,
) -> list[float] | list[list[float]]:
    """OpenAI SDK를 직접 사용하여 임베딩 생성 (encoding_format 지원).

    LangChain의 OpenAIEmbeddings가 encoding_format을 지원하지 않을 때 사용.

    Args:
        model: 임베딩 모델 이름
        encoding_format: 인코딩 형식 ("float")
        input_text: 임베딩할 텍스트 (문자열 또는 문자열 리스트)
        **kwargs: 추가 파라미터

    Returns:
        임베딩 벡터 리스트 (단일 텍스트) 또는 리스트의 리스트 (여러 텍스트)
    """
    from openai import OpenAI

    api_key, base_url = _resolve_api_context()

    client = OpenAI(
        base_url=base_url,
        api_key=api_key,
    )

    # input_text가 비어있으면 kwargs에서 가져오기
    if not input_text:
        input_text = kwargs.get("input", "")

    response = client.embeddings.create(
        model=model,
        input=input_text,
        encoding_format=encoding_format,
    )

    # 단일 텍스트인 경우 첫 번째 임베딩 반환
    if isinstance(input_text, str):
        return response.data[0].embedding
    else:
        # 여러 텍스트인 경우 모든 임베딩 반환
        return [item.embedding for item in response.data]


def get_available_model_types() -> dict[str, list[str]]:
    """OpenRouter에서 사용 가능한 모델 유형을 반환합니다.

    Returns:
        dict[str, list[str]]: 모델 유형별 모델 목록
    """
    return {
        "chat": [
            "openai/gpt-4.1",
            "openai/gpt-4.1-mini",
            "openai/gpt-5",
            "openai/gpt-5-mini",
            "anthropic/claude-sonnet-4.5",
            "anthropic/claude-haiku-4.5",
            "google/gemini-2.5-flash-preview-09-2025",
            "google/gemini-pro-2.5",
            "x-ai/grok-4-fast",
            "moonshotai/kimi-k2-thinking",
            "liquid/lfm-2.2-6b",
            "z-ai/glm-4.6",
        ],
        "embedding": [
            "openai/text-embedding-3-small",
            "openai/text-embedding-3-large",
            "google/gemini-embedding-001",
            "qwen/qwen3-embedding-0.6b",
            "qwen/qwen3-embedding-4b",
            "qwen/qwen3-embedding-8b",
        ],
    }


embeddings = create_embedding_model()
llm = create_openrouter_llm()

In [23]:
from langchain_core.messages import SystemMessage, HumanMessage
# from src.llm.client import create_openrouter_llm

# LLM은 한 번만 생성해서 재사용 (중요)
keyword_llm = create_openrouter_llm(
    model="openai/gpt-oss-20b:free",  # 또는 openai/gpt-4o, gpt-oss-20b 등
    temperature=0
)

In [26]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
아래의 문서를 참고하여 질문에 대한 답변을 작성하세요.

질문: {question}

참고 문서:
{context}

답변:
""")

In [27]:
def ask_question(question: str, retriever):
    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)
    messages = prompt.format_messages(question=question, context=context)
    result = keyword_llm.invoke(messages)
    return result.content

In [ ]:
# def ask_question(question: str, retriever):
#     docs = retriever.invoke(question)
#     context = "\n\n".join(doc.page_content for doc in docs)
#     messages = prompt.format_messages(question=question, context=context)
#     result = llm_model.invoke(messages)
#     return result.content

In [28]:
question = "삼성전자의 사업 영역은?"

In [29]:
print("\n[Chroma 검색 결과]")
print(ask_question(question, chroma_retriever))


[Chroma 검색 결과]
**삼성전자의 사업 영역**

삼성전자는 크게 **3개의 사업 부문**으로 나누어 독립적으로 운영됩니다. 각 부문은 특정 제품군과 기술 영역을 담당하며, 글로벌 시장에서의 경쟁력을 강화하고 있습니다.

| 부문 | 주요 사업 영역 | 대표 제품·서비스 |
|------|----------------|------------------|
| **CE (Consumer Electronics)** | 가전·디스플레이·생활가전 | • 스마트 TV, 초대형 프리미엄 TV<br>• 모니터, 냉장고, 세탁기, 에어컨, 프린터 등<br>• 의료기기(정밀 진단·빠른 검사) |
| **IM (Information Technology & Mobile Communications)** | 모바일·통신·무선 | • 스마트폰, 태블릿, 웨어러블 기기<br>• 무선통신 모듈·네트워크 장비<br>• 5G·6G 등 차세대 통신 기술 |
| **DS (Device Solutions)** | 반도체·디바이스 솔루션 | • 메모리(DRAM, NAND), 시스템 반도체(SoC, 프로세서)<br>• 파운드리·IC 설계·제조<br>• 전력 관리·배터리 솔루션 등 |

### 핵심 포인트
1. **독립 경영**: 각 부문은 자체적인 전략과 재무를 관리하며, 부문별 전문성을 극대화합니다.  
2. **글로벌 리더십**: CE 부문은 스마트TV와 생활가전에서 세계 1위를 목표로 하고, IM 부문은 무선 통신에서 인간 중심의 혁신을 추구합니다. DS 부문은 반도체 분야에서 기술 선도와 생산 규모를 확대하고 있습니다.  
3. **통합 혁신**: 세 부문은 서로 연계된 기술과 제품을 통해 ‘스마트 홈’, ‘IoT’, ‘AI’ 등 새로운 비즈니스 모델을 창출하고 있습니다.

이처럼 삼성전자는 **가전·디스플레이·생활가전, 모바일·통신·무선, 반도체·디바이스 솔루션** 세 가지 핵심 영역을 통해 전 세계 소비자와 기업에 다양한 솔루션을 제공하고 있습니다.


In [30]:
print("\n[BM25 검색 결과]")
print(ask_question(question, bm25_retriever))


[BM25 검색 결과]
**삼성전자의 사업 영역**

- **모바일·가전**  
  - 스마트폰, 태블릿, 스마트워치 등 모바일 기기  
  - TV, 냉장고, 세탁기, 에어컨 등 가전제품  

- **자동차 전장·Connected Car**  
  - 2016년 미국 전장업체 Harman을 인수해 자동차용 인포테인먼트·텔레매틱스 시장에 진입  
  - 향후 10년 내 Connected Car 비중이 90%에 육박할 전망에 따라 ‘스마트카’ Tier‑1 부품 공급망을 목표  

- **기타 핵심 사업**  
  - 반도체(메모리·프로세서 등)  
  - 디스플레이(패널·모듈)  
  - IT·모바일 커뮤니케이션 솔루션  

> 2015년 말 기준 삼성전자의 총자산은 약 169조원, 부채는 32.5조원, 자본총계는 136조원으로, 전 세계적으로 다양한 산업군에 걸친 대규모 기업임을 보여줍니다.


In [31]:
print("\n[ENSEMBLE 검색 결과]")
print(ask_question(question, ensemble_retriever))


[ENSEMBLE 검색 결과]
**삼성전자의 사업 영역**

삼성전자는 크게 **3개의 독립 부문**으로 나누어 운영하며, 각 부문은 아래와 같은 핵심 제품·서비스를 제공합니다.

| 부문 | 주요 사업 영역 | 대표 제품·서비스 |
|------|----------------|------------------|
| **CE (Consumer Electronics)** | 가전·생활가전·디스플레이·의료기기 | 스마트TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등 |
| **IM (Information Technology & Mobile Communications)** | 모바일·무선·통신 | 스마트폰, 무선통신 장비, IoT·스마트폰용 소프트웨어 등 |
| **DS (Device Solutions)** | 반도체·디바이스 솔루션 | DRAM·NAND·SSD·프로세서·전력 관리 IC 등 반도체 제품 |

> **추가 사업 영역**  
> 2020년대 초반 삼성전자는 **Harman 인수**를 통해 **자동차 전장(Connected Car)** 시장에도 진출했습니다. 이로써 차량용 인포테인먼트·텔레매틱스 분야에서 1~2위 수준의 시장 점유율을 확보하고, 향후 10년 내 Connected Car 비중이 90%에 육박할 전망에 따라 자동차 부품·전장 사업도 중요한 성장 동력으로 부각되고 있습니다.

따라서 삼성전자의 사업 영역은 **가전·생활가전·디스플레이·의료기기(CE)**, **모바일·무선·통신(IM)**, **반도체·디바이스 솔루션(DS)** 그리고 **자동차 전장(Connected Car)** 등으로 구성됩니다.
